https://github.com/ilkayDevran/Gaussian_Filtering/tree/456b41df03761546cd61634caaeb7817478dfd47

In [17]:
#import other
%run other.ipynb
%run muti_filter.ipynb

In [18]:
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [23]:
def Main():
    # 匯入圖片，並轉成 array
    image_path = 'source/fume_hood.jpg'
    init_image = other_class.open_image(image_path)
    output_image = np.array(init_image) # array
    #------------------------------裁減圖片--------------------------------------------
    output_image = other_class.cut_image(init_image,300,1300,0,1080) #w_min, w_max, h_min, h_max
    
    #------------------------------二值化(輸入array)--------------------------------------------
    output_image = filter_class.binarization(output_image, threshold_low=30, threshold_high=205 )
    
    #------------------------------中值濾波(輸入array)--------------------------------------------
    #output_image = filter_class.median_filter(output_image, n = 3)
    '''
    n : kernel(默認為3)
    卷積方式 : 在圖片周圍根據kernel增加padding
    '''
    
    #------------------------------均值濾波(輸入array)--------------------------------------------
    #output_image = filter_class.average_filter(output_image, n = 3)
    '''
    n : kernel(默認為3)
    卷積方式 : 在圖片周圍根據kernel增加padding
    '''
    
    #------------------------------二值化(輸入array)--------------------------------------------
    #output_image = filter_class.binarization(output_image, threshold_low=30, threshold_high=225 )
    #output_image = filter_class.average_filter(output_image, n = 3)
    
    #------------------------------銳化濾波(輸入array)--------------------------------------------
    '''
    卷積方式 : 在圖片周圍根據kernel增加padding
    '''
    #output_image = filter_class.Sharpening_filter_laplacian_3x3(output_image, c=1, level=28)
    #output_image = filter_class.Sharpening_filter_laplacian_5x5(output_image)
    #------------------------------對比(histogram equalization)--------------------------------------------
    output_image = filter_class.contrast_histogram_equalization(output_image)
    
    
    # 並將array轉成圖片，並儲存
    show_image = Image.fromarray(output_image, 'RGB')
    show_image.save('result/new_cat.jpg')

In [24]:
if __name__ == '__main__':
    Main()

影像大小 : (1920, 1080)
